In [2]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 📌 Configuração do Brave
brave_path = "C:/Program Files/BraveSoftware/Brave-Browser/Application/brave.exe"
options = webdriver.ChromeOptions()
options.binary_location = brave_path
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--start-maximized")  # Abre em tela cheia

# 📌 Inicializa o WebDriver
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager(path=r"C:\chromedriver").install()),
    options=options
)

# 📌 Acessa o site
url = "https://www.buyfromportugal.com/export/"
driver.get(url)
time.sleep(5)

# 📌 Aceita os cookies (se existir o botão)
try:
    cookie_button = driver.find_element(By.ID, "onetrust-accept-btn-handler")
    if cookie_button.is_displayed():
        cookie_button.click()
        print("✅ Botão de cookies clicado.")
except NoSuchElementException:
    print("⚠️ Botão de cookies não encontrado. Continuando execução.")

# 📌 Pesquisa no campo "Sector"
try:
    sector_input = driver.find_element(By.ID, "keyword_sector")
    sector_input.send_keys("1906")
    time.sleep(3)

    # 📌 Clica na sugestão
    try:
        suggestion_list = driver.find_element(By.XPATH, '//*[@id="eac-container-keyword_sector"]/ul')
        first_suggestion = suggestion_list.find_element(By.TAG_NAME, "li")
        first_suggestion.click()
        print("✅ Sugestão clicada com sucesso!")
    except NoSuchElementException:
        print("⚠️ Nenhuma sugestão encontrada!")

    # 📌 Clica no botão "Search"
    search_button = driver.find_element(By.ID, "pesquisar_btn")
    search_button.click()
    time.sleep(5)  # Aguarda carregar os resultados
except Exception as e:
    print(f"❌ Erro ao interagir com a pesquisa: {e}")

# 📌 Loop para percorrer todas as páginas
page = 1
while True:
    print(f"\n🔄 Extraindo dados da página {page}...")

    # 📌 Extrai nomes e links das empresas
    try:
        companies = driver.find_elements(By.CLASS_NAME, "search-results-view-detail")

        if not companies:
            print("⚠️ Nenhuma empresa encontrada nesta página!")

        else:
            print(f"✅ {len(companies)} empresas encontradas na página {page}:\n")

            for company in companies:
                try:
                    name = company.text.strip()
                    link = company.get_attribute("href")
                    print(f"📌 {name} - {link}")
                except Exception as e:
                    print(f"❌ Erro ao extrair informações da empresa: {e}")

    except Exception as e:
        print(f"❌ Erro ao encontrar empresas: {e}")

    # 📌 Verifica se há uma próxima página
    try:
        next_button = driver.find_element(By.XPATH, '//a[contains(@class, "paginate_button") and contains(text(), "{}")]'.format(page + 1))
        
        if next_button.is_displayed():
            next_button.click()
            page += 1
            time.sleep(5)  # Aguarda a página carregar
        else:
            break  # Sai do loop se não houver mais páginas
    except NoSuchElementException:
        print("✅ Fim da paginação. Todas as empresas foram extraídas.")
        break

# 📌 Fecha o navegador
driver.quit()




TypeError: ChromeDriverManager.__init__() got an unexpected keyword argument 'path'